# Preparación datos para introducir al modelo

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col, regexp_extract, count

# Detener sesión anterior
try:
    spark.stop()
except:
    pass

spark = SparkSession.builder \
    .appName("Preparación dataset para entrenamiento del modelo predictivo") \
    .config("spark.master", "local[*]") \
    .config("spark.driver.memory", "20g") \
    .config("spark.executor.memory", "20g") \
    .getOrCreate()

In [ ]:
ruta_dataset = "data/resultados/dataset_final.parquet"
ruta_rules = "data/resultados/fpgrowth_rules.parquet"
ruta_arbol = "data/resultados/dataset_arbol.parquet"

In [ ]:
# Leer dataset final (original con todos los datos de ingresos)
df_dataset = spark.read.parquet(ruta_dataset)
print("Dataset original cargado correctamente.")
df_dataset.printSchema()
df_dataset.show(5)

In [ ]:
df_dataset.filter(col("edad") == 91).show()

In [ ]:
# Leer dataset reglas
df_rules = spark.read.parquet(ruta_rules)
print("Dataset reglas cargado correctamente.")
df_rules.printSchema()
df_rules.show(5)

In [ ]:
# Leer dataset arbol
df_arbol = spark.read.parquet(ruta_arbol)
print("Dataset para el modelo predictivo cargado correctamente.")
df_arbol.printSchema()
df_arbol.show(5)

In [ ]:
# Verificar registros de cada dataset
print(f" Registros en DATASET FINAL: {df_dataset.count()}")
print(f" Registros en DATASET REGLAS GENERADAS: {df_rules.count()}")
print(f" Registros en DATASET ARBOL: {df_arbol.count()}")

In [ ]:
len(df_arbol.columns)

# Análisis exploratorio de los datos del arbol de decisiones

In [ ]:
from pyspark.sql.functions import explode, col

# Extraer los diagnósticos únicos desde la columna "dominios"
df_diagnosticos = df_arbol.select(explode(col("dominios")).alias("diagnostico")).distinct()

# Contar la cantidad de diagnósticos únicos
num_diagnosticos = df_diagnosticos.count()
print(f"Total de diagnósticos únicos: {num_diagnosticos}")

In [ ]:
from pyspark.sql.functions import size

# Contar cuántos diagnósticos tiene cada paciente
df_pacientes_diagnosticos = df_arbol.withColumn("num_diagnosticos", size(col("dominios")))

# Mostrar estadísticas generales
df_pacientes_diagnosticos.select("num_diagnosticos").describe().show()

In [ ]:
# Crear una nueva columna con el número de diagnósticos por paciente
df_arbol = df_arbol.withColumn("num_diagnosticos", size("dominios"))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Extraer los datos de la columna "num_diagnosticos"
num_diagnosticos = df_arbol.select("num_diagnosticos").rdd.flatMap(lambda x: x).collect()

# Crear el histograma
plt.figure(figsize=(10, 6))
sns.histplot(num_diagnosticos, bins=30, kde=True, color='#99CC99', alpha=0.7, edgecolor='white')

# Configurar etiquetas y título
plt.xlabel("Número de Diagnósticos por Paciente")
plt.ylabel("Frecuencia")
plt.title("Distribución del Número de Diagnósticos por Paciente")

# Mostrar el gráfico
plt.show()

El histograma muestra la distribución del número de diagnósticos por paciente, evidenciando que la mayoría de los pacientes tienen entre 5 y 15 diagnósticos. Se observa una ligera asimetría a la derecha, lo que indica que hay pacientes con una cantidad de diagnósticos significativamente mayor al promedio. Además, la curva KDE refuerza la presencia de ciertos valores más frecuentes, con un pico alrededor de 10 diagnósticos.

Este análisis nos sugiere que podría ser útil agrupar a los pacientes según su número de diagnósticos para definir una métrica de complejidad diagnóstica.

In [ ]:
from pyspark.sql.functions import col, when, expr

# Obtener los cuantiles del número de diagnósticos
quantiles = df_arbol.approxQuantile("num_diagnosticos", [0.25, 0.5, 0.75, 0.9], 0.01)

# Definir los umbrales
q25, q50, q75, q90 = quantiles

# Crear la columna de complejidad diagnóstica basada en los cuantiles
df_arbol = df_arbol.withColumn(
    "complejidad_diagnostica",
    when(col("num_diagnosticos") <= q25, "Baja")
    .when((col("num_diagnosticos") > q25) & (col("num_diagnosticos") <= q75), "Media")
    .when((col("num_diagnosticos") > q75) & (col("num_diagnosticos") <= q90), "Alta")
    .otherwise("Muy Alta")
)

df_arbol.groupBy("complejidad_diagnostica").count().show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Convertir los datos a Pandas para graficar
df_complejidad_pandas = df_arbol.groupBy("complejidad_diagnostica").count().toPandas()

# Ordenar categorías en el gráfico
orden_categorias = ["Baja", "Media", "Alta", "Muy Alta"]
df_complejidad_pandas = df_complejidad_pandas.set_index("complejidad_diagnostica").loc[orden_categorias].reset_index()

# Configurar el estilo de la visualización
plt.figure(figsize=(8,5))
sns.barplot(
    data=df_complejidad_pandas,
    x="complejidad_diagnostica",
    y="count",
    alpha=0.7,
    edgecolor='black',
    color="#99CC99"
)

# Etiquetas y título
plt.xlabel("Nivel de Complejidad Diagnóstica", fontsize=12)
plt.ylabel("Número de Pacientes", fontsize=12)
plt.title("Distribución de Pacientes por Complejidad Diagnóstica", fontsize=14)
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Lista de variables categóricas a visualizar
variables_categoricas = ["edad_categoria", "sexo", "estado_civil", "tipo_seguro", "grupo_poblacional", "muerte_durante_ingreso"]

# Convertir el DataFrame de Spark a Pandas (solo con las columnas necesarias)
df_pandas = df_arbol.select(variables_categoricas).toPandas()

# Configuración de los gráficos
fig, axes = plt.subplots(2, 3, figsize=(18, 12))  
axes = axes.flatten()

# Generar gráficos de barras para cada variable categórica
for i, var in enumerate(variables_categoricas):
    df_pandas[var].value_counts().plot(kind="bar", ax=axes[i], alpha=0.7, edgecolor="white")
    axes[i].set_title(f"Distribución de {var}")
    axes[i].set_ylabel("Frecuencia")
    axes[i].set_xlabel(var)
    axes[i].tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

## Modelo XGBoost para un problema *multilabel*

En este dataset, cada observación puede estar asociada simultáneamente a múltiples etiquetas, lo que es especialmente relevante en el contexto clínico analizado, donde un mismo paciente puede presentar diagnósticos pertenecientes a distintos dominios ICD-10. Para abordar este problema, es necesario aplicar técnicas específicas de clasificación multilabel que permitan al modelo aprender las relaciones entre las características (parámetros bioquímicos y otras variables) y múltiples etiquetas (dominios o capítulos de diagnóstico). Este enfoque es clave para no perder información relevante en el proceso de predicción.

La ha elegido XGBoost como algoritmo principal por varias razones. En primer lugar, XGBoost admite valores faltantes de manera nativa, lo que elimina la necesidad de imputar los valores NaN en los datos de entrada. En segundo lugar, su capacidad para manejar directamente problemas multilabel permite entrenar un modelo eficiente que tenga en cuenta la posible relación entre las etiquetas. Además, su enfoque basado en árboles de decisión es robusto frente a datos tabulares como el que se analiza, con características heterogéneas y correlacionadas.

El modelo generado será capaz de predecir múltiples capítulos de diagnóstico para un paciente dado.

Este enfoque permitirá una clasificación más precisa y granular, facilitando aplicaciones prácticas como la predicción de patrones de diagnóstico, la identificación de combinaciones atípicas de capítulos, y la mejora en la toma de decisiones clínicas. El procedimiento descrito no solo es una solución técnicamente adecuada para este problema, sino que también alinea las características del dataset con un enfoque robusto y escalable. La aplicación de un modelo multilabel basado en XGBoost permitirá obtener resultados precisos y relevantes, maximizando el aprovechamiento de la información disponible en los datos.

In [ ]:
df_arbol.show(5)

In [ ]:
estados_civiles = df_arbol.select("estado_civil").distinct().rdd.flatMap(lambda x: x).collect()

In [ ]:
tipos_seguro = df_arbol.select("tipo_seguro").distinct().rdd.flatMap(lambda x: x).collect()

In [ ]:
grupos_poblacionales = df_arbol.select("grupo_poblacional").distinct().rdd.flatMap(lambda x: x).collect()

In [ ]:
edad_categorias = df_arbol.select("edad_categoria").distinct().rdd.flatMap(lambda x: x).collect()

In [ ]:
from pyspark.sql.functions import col, when, array, struct, lit, expr
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml import Pipeline

# Crear una copia del DataFrame original antes de hacer modificaciones
df_procesado = df_arbol

# Eliminar las variables que no se van a introducir en el modelo
columnas_a_eliminar = ["id_ingreso"]
df_procesado = df_procesado.drop(*columnas_a_eliminar)

# Convertir "sexo" a numérico (0 = M, 1 = F)
df_procesado = df_procesado.withColumn("sexo", when(col("sexo") == "F", 1).otherwise(0))

# Convertir "muerte_durante_ingreso" a tipo numérico
df_procesado = df_procesado.withColumn("muerte_durante_ingreso", col("muerte_durante_ingreso").cast("integer"))


# Definir las columnas categóricas y sus valores únicos
columnas_categoricas = {
    "estado_civil": estados_civiles,
    "tipo_seguro": tipos_seguro,
    "edad_categoria": edad_categorias,
    "grupo_poblacional": grupos_poblacionales
}

# Generar expresiones para las nuevas columnas
df_procesado = df_procesado.withColumns(
    {f"{col}_{val}": expr(f"IF({col} = '{val}', 1, 0)") for col, vals in columnas_categoricas.items() for val in vals}
)

df_procesado = df_procesado.drop('estado_civil', 'tipo_seguro', 'grupo_poblacional', 'complejidad_diagnostica', 'edad_categoria')
    
# Verificar el esquema del DataFrame después de la transformación
df_procesado.printSchema()
df_procesado.show(5)

Para el preprocesamiento de los datos para su uso en modelos de aprendizaje automático, se han eliminado variables irrelevantes, convertido variables categóricas en un formato numérico mediante One-Hot Encoding, y normalizado las variables binarias. Se ha utilizado un Pipeline para aplicar las transformaciones de manera eficiente en PySpark, asegurando que el dataset esté correctamente estructurado antes de entrenar el modelo.

In [ ]:
from pyspark.sql.functions import explode, col

# Obtener la frecuencia de cada diagnóstico
df_freq = df_procesado.select(explode(col("dominios")).alias("diagnostico"))
df_freq = df_freq.groupBy("diagnostico").count()

# Filtrar los diagnósticos que aparecen al menos en 200 pacientes
df_freq = df_freq.filter(col("count") >= 200)

# Obtener la lista de diagnósticos más frecuentes
diagnosticos_frecuentes = [row["diagnostico"] for row in df_freq.collect()]
print(f"Número de diagnósticos después del filtrado: {len(diagnosticos_frecuentes)}")


In [ ]:
df_procesado.select("dominios").show()

In [ ]:
# Binarizar los diagnosticos
from pyspark.sql.functions import expr

# Conservar las columnas existentes sin "dominios"
columnas_existentes = [col(c) for c in df_procesado.columns if c != "dominios"]

# Generar las nuevas columnas binarias
columnas_diagnosticos = [expr(f"IF(array_contains(dominios, '{diag}'), 1, 0) AS `{diag}`") for diag in diagnosticos_frecuentes]

# Aplicar la transformación de manera eficiente en una sola operación
df_procesado = df_procesado.select(*columnas_existentes, *columnas_diagnosticos)

# Eliminar la columna original "dominios"
df_procesado = df_procesado.drop("dominios")

# Verificar la estructura final
df_procesado.printSchema()
df_procesado.show(5)

In [ ]:
# Obtener todas las columnas del DataFrame
columnas = df_procesado.columns

# Verificar si hay nombres duplicados
from collections import Counter
contador_columnas = Counter(columnas)
columnas_duplicadas = [col for col, count in contador_columnas.items() if count > 1]

print(f"Columnas duplicadas: {columnas_duplicadas}")

In [ ]:
ruta_guardado = "data/resultados/arbol_preprocesado.parquet"
df_procesado.write.mode("overwrite").parquet(ruta_guardado)

print("Dataset guardado")